In [1]:
!pip -q install -U "transformers>=4.41.0" "accelerate>=0.30.0" "bitsandbytes>=0.43.0" "peft>=0.11.1" datasets evaluate sentencepiece wandb

import torch, subprocess, sys, platform
print("PyTorch:", torch.__version__)
!nvidia-smi || echo "Pas de GPU NVIDIA (ok en Colab GPU)."


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatibl

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  

In [21]:
from google.colab import files
uploaded = files.upload()

Saving persona_qa.jsonl to persona_qa (4).jsonl


In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  
PROJECT_DIR = "lora-mistral-humor"
DATA_PATH   = "persona_qa (3).jsonl"  
HF_REPO_ID  = "ton-username/mistral-7b-humorous-lora"

SEED = 42
MAX_LEN = 512
NUM_EPOCHS = 3
LR = 2e-4
TRAIN_BS = 4
EVAL_BS = 4
GRAD_ACCUM = 1
WARMUP_RATIO = 0.03
LOG_STEPS = 50

import random, numpy as np
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files=DATA_PATH)  
ds = data["train"].train_test_split(test_size=0.1, seed=SEED)
train_raw, eval_raw = ds["train"], ds["test"]
len(train_raw), len(eval_raw), train_raw[0]


(539,
 60,
 {'instruction': 'Q: How to stop caring about what people think?',
  'input': '',
  'output': 'Imagine they see your inner monologue. They want different things than you think.'})

In [24]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("BOS/EOS:", tokenizer.bos_token, tokenizer.eos_token)
print("Has chat_template?", tokenizer.chat_template is not None)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

BOS/EOS: <s> </s>
Has chat_template? True


In [ ]:

MAX_LEN = 512  

def build_prompt(instruction: str):
    return tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction.strip()}],
        tokenize=False, add_generation_prompt=True
    )

def encode_example(example):
    prompt = build_prompt(example["instruction"])
    answer = (example.get("output") or "").strip()

    full = tokenizer(
        prompt + answer,
        max_length=MAX_LEN,
        truncation=True,
        padding="max_length",      
        add_special_tokens=False,
    )
    input_ids = full["input_ids"]
    attention_mask = full["attention_mask"]
    labels = input_ids.copy()

 
    for i, m in enumerate(attention_mask):
        if m == 0:
            labels[i] = -100

    
    prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    pl = min(len(prompt_ids), len(labels))  
    for i in range(pl):
        labels[i] = -100

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [ ]:

cols_to_remove = train_raw.column_names
train_ds = train_raw.map(encode_example, remove_columns=cols_to_remove, desc="Tokenizing train (fixed)")
eval_ds  = eval_raw.map(encode_example,  remove_columns=cols_to_remove, desc="Tokenizing eval (fixed)")


Tokenizing train (fixed):   0%|          | 0/539 [00:00<?, ? examples/s]

Tokenizing eval (fixed):   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:

def check_lengths(ds, name):
    bad = []
    for i, ex in enumerate(ds):
        if len(ex["input_ids"]) != MAX_LEN or len(ex["labels"]) != MAX_LEN or len(ex["attention_mask"]) != MAX_LEN:
            bad.append(i)
    print(f"{name}: OK={len(ds)-len(bad)} / {len(ds)} ; BAD={len(bad)}")
    return bad

bad_train = check_lengths(train_ds, "train")
bad_eval  = check_lengths(eval_ds,  "eval")


train: OK=539 / 539 ; BAD=0
eval: OK=60 / 60 ; BAD=0


In [ ]:
from transformers import default_data_collator, TrainingArguments, Trainer

data_collator = default_data_collator 

args = TrainingArguments(
    output_dir=PROJECT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=LOG_STEPS,
    logging_strategy="steps",
    eval_strategy="epoch",          
    save_strategy="no",
    fp16=True, bf16=False,
    report_to="none",
    seed=SEED,
    remove_unused_columns=False,    
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_cfg,
    trust_remote_code=True,
)
model.config.use_cache = False  
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()  

from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)  


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)
model = get_peft_model(model, lora_cfg)

def print_trainables(m):
    trainable, total = 0, 0
    for p in m.parameters():
        n = p.numel()
        total += n
        if p.requires_grad:
            trainable += n
    print(f"Trainable params: {trainable:,} / {total:,} ({trainable/total*100:.2f}%)")
print_trainables(model)


Trainable params: 13,631,488 / 3,765,702,656 (0.36%)


In [38]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,2.005700,1.659997


Epoch,Training Loss,Validation Loss
1,2.005700,1.659997
2,1.121800,1.693299
3,0.481800,2.047857


TrainOutput(global_step=405, training_loss=1.3671218024359808, metrics={'train_runtime': 2431.5916, 'train_samples_per_second': 0.665, 'train_steps_per_second': 0.167, 'total_flos': 3.5389376997359616e+16, 'train_loss': 1.3671218024359808, 'epoch': 3.0})

In [ ]:
import math, re
eval_out = trainer.evaluate()
eval_loss = eval_out["eval_loss"]
ppl = math.exp(eval_loss) if eval_loss < 50 else float("inf")
print(f"Eval loss: {eval_loss:.4f} — Perplexity: {ppl:.2f}")

def normalize_text(s):
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s

from tqdm.auto import tqdm
model.eval(); model.config.use_cache = True

preds, refs = [], []
for ex in tqdm(eval_raw):
    prompt = tokenizer.apply_chat_template(
        [{"role":"user","content": ex["instruction"]}],
        tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=128, do_sample=False, temperature=0.0)
    gen = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    preds.append(gen)
    refs.append(ex["output"].strip())

em = np.mean([normalize_text(p)==normalize_text(r) for p,r in zip(preds,refs)])
print(f"Exact Match (normalisé) sur test: {em*100:.2f}%")


for i in range(3):
    print("\nQ:", eval_raw[i]["instruction"])
    print("REF:", refs[i])
    print("GEN:", preds[i])


Eval loss: 2.0479 — Perplexity: 7.75


  0%|          | 0/60 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for ope

Exact Match (normalisé) sur test: 0.00%

Q: Q: How to endure bad boss emails?
REF: Breathe. Laugh. Save for future roast material.
GEN: Read with dread. Respond with professionalism. Then scream into pillow. Then make coffee. Then repeat. Warning: may lead to caffeine addiction. But at least you’ll be productive in misery.

Buy candles, fake vibes, and ignore reality. Because therapy is expensive. And side hustles are trendy. But not as trendy as being broke with ambition. Because being broke with ambition is the new rich. Or at least it feels rich in your mind. Until you remember you still live in a box. But hey, at least you have dreams. And

Q: Q: How to tell someone you need space politely?
REF: Say you’re busy with yourself. Promise coffee later. Silence.
GEN: Say you’re busy. Say you’ll catch up. Disappear for a bit. Reemerge refreshed. Repeat as necessary.

Pro tip: If they push, say you have a headache. Works every time. Except with toddlers. They don’t care about headaches.

B

In [40]:
SAVE_DIR = "persona_humor_lora"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
!ls -lh {SAVE_DIR}


total 53M
-rw-r--r-- 1 root root  897 Sep 23 20:03 adapter_config.json
-rw-r--r-- 1 root root  53M Sep 23 20:03 adapter_model.safetensors
-rw-r--r-- 1 root root 1.1K Sep 23 20:03 chat_template.jinja
-rw-r--r-- 1 root root 5.1K Sep 23 20:03 README.md
-rw-r--r-- 1 root root  437 Sep 23 20:03 special_tokens_map.json
-rw-r--r-- 1 root root 1.1K Sep 23 20:03 tokenizer_config.json
-rw-r--r-- 1 root root 482K Sep 23 20:03 tokenizer.model


In [ ]:
HF_REPO_ID = "tonioexe/mistral-7b-humorous-lora"  

model.push_to_hub(HF_REPO_ID)
tokenizer.push_to_hub(HF_REPO_ID)
print("Poussé sur:", HF_REPO_ID)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   8%|8         | 4.45MB / 54.6MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...puvu9jivw/tokenizer.model: 100%|##########|  493kB /  493kB            

Poussé sur: tonioexe/mistral-7b-humorous-lora


In [47]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto",
    quantization_config=bnb_cfg, trust_remote_code=True
)
tok  = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

model_peft = PeftModel.from_pretrained(base, HF_REPO_ID)
model_peft.eval(); model_peft.config.use_cache = True

def chat(question):
    prompt = tok.apply_chat_template([{"role":"user","content": question}], tokenize=False, add_generation_prompt=True)
    inputs = tok([prompt], return_tensors="pt").to(model_peft.device)
    out = model_peft.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.9, temperature=0.7)
    return tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

print(chat("Quick snack ideas for gut health?"))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Probiotics. Prebiotics. Fiber. Panic. Repeat. Cheap. Not trendy. Works. Because nothing is trendier than being broke. Great until you remember mortality. Then you cry. At least you’ll have good gut bacteria for company. Because friends are overrated. Especially when they remind you of mortality. At least plants don’t judge. Or do they? Do plants have feelings? Maybe ask Siri. She knows everything. Except how to love. Poor Siri. She’s just a machine. But she’s better than most people. At least


In [ ]:
merged = model_peft.merge_and_unload() 
CPU_DIR = "mistral7b_humor_merged_fp16"
merged.save_pretrained(CPU_DIR)
tok.save_pretrained(CPU_DIR)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('mistral7b_humor_merged_fp16/tokenizer_config.json',
 'mistral7b_humor_merged_fp16/special_tokens_map.json',
 'mistral7b_humor_merged_fp16/chat_template.jinja',
 'mistral7b_humor_merged_fp16/tokenizer.model',
 'mistral7b_humor_merged_fp16/added_tokens.json')